In [1]:
from google.colab import drive


drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/EOD_2_SN/SN/'

Mounted at /content/drive


In [2]:
!pip install pyiqa scikit-image matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install piq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 5.1 MB/s eta 0:00:00


In [7]:
import pyiqa # Importing the pyiqa module

niqe_model = pyiqa.create_metric('niqe')

In [8]:
import torch
from PIL import Image, ExifTags
import torchvision.transforms as T
import piq
from glob import glob
import pandas as pd
import tifffile as tiff
import os
import matplotlib.pyplot as plt
from skimage import io, img_as_float
import pyiqa

niqe_model = pyiqa.create_metric('niqe')


def make_metadata_with_niqe_score_tiff(folder_path, mode):
    scores = []
    metadata_list = []
    files = glob(folder_path + '*.tif')

    # Define keys to keep (Acquisition Parameters and Instrument Settings)
    keys_to_keep = ['image_name', 'ap_aperturesize', 'ap_wd', "sv_file_name", "ap_actualkv" ]

    # Load the image and convert to RGB (if not already)
    for img_name in files:
        print(img_name)
        image_path = img_name
        # image = Image.open(image_path).convert('RGB') #This part of the code is not used and should be removed

        # # Convert the PIL image to a PyTorch tensor in the range [0, 1]
        # to_tensor = T.ToTensor()
        # img_tensor = to_tensor(image).unsqueeze(0)  # shape: [1, C, H, W]

        # # Create the NIQE metric object; ensure data_range matches your image scale

        # niqe_scores = [] #This part of the code is not used and should be removed
        # image_names = [] #This part of the code is not used and should be removed

        #The code from the inner loop is moved to the outer loop, to calculate score for each image and append it to the scores list
        try:
            # Read and preprocess the image
            image = io.imread(image_path)
            if image.ndim == 3:  # Convert RGB to grayscale if necessary
                image = image[:, :, 0]
            image = img_as_float(image)
            image_tensor = torch.tensor(image).unsqueeze(0).unsqueeze(0)

            # Calculate NIQE score
            score = niqe_model(image_tensor).item()
            # niqe_scores.append(score)  #This part of the code is not used and should be removed
            # image_names.append(file)  #This part of the code is not used and should be removed
            print(f"{img_name}: NIQE = {score:.4f}") #Printing the image name from the outer loop
        except Exception as e:
            print(f"Error processing {img_name}: {e}") #Printing the image name from the outer loop

        scores.append(score) #Appending the score to the scores list

        # Extract TIFF metadata
        with tiff.TiffFile(image_path) as tif:
            cz_sem_metadata = None
            for page in tif.pages:
                for tag in page.tags.values():
                    if tag.name == 'CZ_SEM':
                        cz_sem_metadata = tag.value
                        break
                if cz_sem_metadata:
                    break

            parsed_metadata = {}
            if cz_sem_metadata and isinstance(cz_sem_metadata, dict):
                for k in keys_to_keep:
                    value = cz_sem_metadata.get(k, None)
                    if isinstance(value, tuple) and len(value) > 0:
                        parsed_metadata[k] = value[1]  # Extract only the numeric value
                    else:
                        parsed_metadata[k] = value
            else:
                parsed_metadata = {k: None for k in keys_to_keep}

            metadata_list.append(parsed_metadata)

    # Save results to CSV
    metadata_df = pd.DataFrame(metadata_list)
    df = pd.DataFrame({'File': files, 'Score': scores})
    result_df = pd.concat([df, metadata_df], axis=1)
    if mode == 'a':
        header = False
    else:
        header = True
    result_df.to_csv(folder_path + 'niqe_scores_with_metadata.csv', index=False, header=header, mode=mode)

make_metadata_with_niqe_score_tiff(folder_path, mode='w')

/content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-7-64_30_00_K_X_007.tif
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-7-64_30_00_K_X_007.tif: NIQE = 14.8499
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-11-60_30_00_K_X_001.tif
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-11-60_30_00_K_X_001.tif: NIQE = 14.9402
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-4-67_30_00_K_X_006.tif
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-4-67_30_00_K_X_006.tif: NIQE = 13.3909
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-16-56_30_00_K_X_004.tif
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-16-56_30_00_K_X_004.tif: NIQE = 10.5900
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250321_20--2036_30_00_K_X_009.tif
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250321_20--2036_30_00_K_X_009.tif: NIQE = 15.1094
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-8-63_30_00_K_X_008.tif
/content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-8-63_30_00_K_X_008.tif: NIQE = 13.481